In [ ]:
# Ch12-4 Gene Discovery

In [ ]:
! pip install biopython pandas 

In [ ]:
#!/usr/bin/env python3
"""
Subtractive Comparative Genomics Pipeline

This script demonstrates a workflow for identifying unique genes in a target organism
by comparing against reference genomes. The pipeline includes:
1. Genome sequence retrieval
2. Gene prediction
3. Sequence similarity comparison
4. Filtering for unique genes
5. Functional annotation of candidate genes

Requirements:
- Biopython
- BLAST+ command line tools
- Prodigal (for prokaryotic gene prediction)
- HMMER and Pfam database (for functional annotation)
"""

In [ ]:
# Import Libraries
import os
import subprocess
from Bio import SeqIO
from Bio.Blast.Applications import NcbiblastpCommandline
import pandas as pd
import argparse

In [ ]:
# Set up Entrez
from Bio import Entrez
Entrez.email = "your.email@example.com"  # you can replace this with your email address

In [ ]:
def setup_directories():
    """Create necessary directories for the pipeline"""
    directories = ['genomes', 'predictions', 'blast_results', 'unique_genes', 'annotations']
    for directory in directories:
        os.makedirs(directory, exist_ok=True)
    return directories

def download_genome(accession, output_dir):
    """Download genome from NCBI using accession number"""
    output_file = f"{output_dir}/{accession}.fasta"
    if not os.path.exists(output_file):
        print(f"Downloading {accession}...")
        # We use Entrez from Biopython to download the genomes
        handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
        with open(output_file, 'w') as out_f:
            out_f.write(handle.read())
        print(f"Downloaded {accession} to {output_file}")
    else:
        print(f"Genome {accession} already exists at {output_file}")
    return output_file

def predict_genes(genome_file, output_dir, organism_type="prokaryote"):
    """Predict genes from genome sequence"""
    genome_name = os.path.basename(genome_file).split('.')[0]
    output_prefix = f"{output_dir}/{genome_name}"
    protein_file = f"{output_prefix}_proteins.faa"
    
    if not os.path.exists(protein_file):
        print(f"Predicting genes for {genome_name}...")
        if organism_type == "prokaryote":
            # Using Prodigal for prokaryotic gene prediction
            cmd = f"prodigal -i {genome_file} -a {protein_file} -o {output_prefix}_genes.gff -f gff"
            subprocess.run(cmd, shell=True, check=True)
        else:
            # For eukaryotes, more complex gene prediction tools would be used
            # This is simplified for demonstration
            print("Eukaryotic gene prediction requires tools like Augustus or MAKER")
        print(f"Gene prediction complete for {genome_name}")
    else:
        print(f"Gene predictions already exist for {genome_name}")
    
    return protein_file

def create_blast_database(protein_file):
    """Create BLAST database from protein sequences"""
    db_name = protein_file
    cmd = f"makeblastdb -in {protein_file} -dbtype prot -out {db_name}"
    subprocess.run(cmd, shell=True, check=True)
    return db_name

def run_blast_comparison(query_proteins, subject_db, output_file, evalue=1e-5):
    """Run BLAST to compare query proteins against subject database"""
    if not os.path.exists(output_file):
        print(f"Running BLAST comparison: {query_proteins} vs {subject_db}")
        blastp_cline = NcbiblastpCommandline(
            query=query_proteins,
            db=subject_db,
            out=output_file,
            outfmt="6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore",
            evalue=evalue,
            max_target_seqs=1
        )
        stdout, stderr = blastp_cline()
        print(f"BLAST comparison complete. Results saved to {output_file}")
    else:
        print(f"BLAST results already exist at {output_file}")
    return output_file

def identify_unique_genes(target_protein_file, blast_results_files, output_dir, identity_threshold=30, coverage_threshold=50):
    """Identify genes in target organism that have no significant hits in reference organisms"""
    target_name = os.path.basename(target_protein_file).split('_')[0]
    
    # Create a dictionary of all proteins in the target organism
    target_proteins = {}
    for record in SeqIO.parse(target_protein_file, "fasta"):
        target_proteins[record.id] = record
    
    # Track proteins with significant hits in reference genomes
    proteins_with_hits = set()
    
    # Process each BLAST result file
    for blast_file in blast_results_files:
        with open(blast_file, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                query_id = parts[0]
                identity = float(parts[2])
                alignment_length = float(parts[3])
                query_length = len(target_proteins[query_id].seq)
                coverage = (alignment_length / query_length) * 100
                
                # If a protein has a significant hit, add it to the set
                if identity >= identity_threshold and coverage >= coverage_threshold:
                    proteins_with_hits.add(query_id)
    
    # Identify proteins without significant hits (unique genes)
    unique_proteins = set(target_proteins.keys()) - proteins_with_hits
    
    # Write unique proteins to output file
    output_file = f"{output_dir}/{target_name}_unique_proteins.faa"
    with open(output_file, 'w') as out_f:
        for protein_id in unique_proteins:
            SeqIO.write(target_proteins[protein_id], out_f, "fasta")
    
    print(f"Identified {len(unique_proteins)} unique proteins in {target_name}")
    print(f"Unique proteins saved to {output_file}")
    
    return output_file, unique_proteins

def annotate_unique_genes(unique_proteins_file, output_dir):
    """Annotate unique genes using HMMER and Pfam database"""
    output_name = os.path.basename(unique_proteins_file).split('_')[0]
    hmmer_output = f"{output_dir}/{output_name}_pfam_annotations.txt"
    
    # In a real implementation, run HMMER against Pfam database
    cmd = f"hmmscan --domtblout {hmmer_output} pfam/Pfam-A.hmm {unique_proteins_file}"
    # subprocess.run(cmd, shell=True, check=True)
    
    # This is a placeholder for parsing HMMER output
    # In a real implementation, parse the domtblout file to extract domain annotations
    annotations = {}
    print(f"Functional annotation complete. Results saved to {hmmer_output}")
    
    return hmmer_output, annotations

In [ ]:
def main():
    # Example usage with Mycobacterium tuberculosis as target and related species as references
    
    # 1. Set up directories
    directories = setup_directories()
    
    # 2. Define target and reference organisms
    target_accession = "NC_000962"  # M. tuberculosis H37Rv
    reference_accessions = [
        "NC_002945",  # M. bovis
        "NC_008769",  # M. avium
        "NC_002677"   # M. leprae
    ]
    
    # 3. Download genomes
    target_genome = download_genome(target_accession, directories[0])
    reference_genomes = [download_genome(acc, directories[0]) for acc in reference_accessions]
    
    # 4. Predict genes
    target_proteins = predict_genes(target_genome, directories[1])
    reference_proteins = [predict_genes(genome, directories[1]) for genome in reference_genomes]
    
    # 5. Create BLAST databases for reference genomes
    reference_dbs = [create_blast_database(proteins) for proteins in reference_proteins]
    
    # 6. Run BLAST comparisons
    blast_results = []
    for i, db in enumerate(reference_dbs):
        ref_name = os.path.basename(reference_genomes[i]).split('.')[0]
        target_name = os.path.basename(target_genome).split('.')[0]
        output_file = f"{directories[2]}/{target_name}_vs_{ref_name}.blast"
        blast_results.append(run_blast_comparison(target_proteins, db, output_file))
    
    # 7. Identify unique genes
    unique_genes_file, unique_gene_ids = identify_unique_genes(
        target_proteins, blast_results, directories[3]
    )
    
    # 8. Annotate unique genes
    annotation_file, annotations = annotate_unique_genes(unique_genes_file, directories[4])
    
    # 9. Generate summary report
    print("\nSubtractive Comparative Genomics Results:")
    print(f"Target organism: {target_accession}")
    print(f"Reference organisms: {', '.join(reference_accessions)}")
    print(f"Total predicted proteins in target: {len(list(SeqIO.parse(target_proteins, 'fasta')))}")
    print(f"Number of unique proteins identified: {len(unique_gene_ids)}")
    print(f"Unique protein sequences saved to: {unique_genes_file}")
    print(f"Functional annotations saved to: {annotation_file}")
    
    # In a complete implementation, additional analyses could include:
    # - GO term enrichment
    # - Pathway analysis
    # - Structural prediction
    # - Phylogenetic analysis of unique genes

if __name__ == "__main__":
    main()

In [ ]:
## AI Tip - Prompt: Update code to actually run HMMER ##
# This was done using Claude

In [ ]:
# First - Download PFAM database
# Create a directory for Pfam
! mkdir -p ~/pfam

# Download the Pfam database (this is a large file, ~1-2GB)
! wget ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/Pfam-A.hmm.gz -P ~/pfam

# Uncompress the file
! gunzip ~/pfam/Pfam-A.hmm.gz

# Press (index) the database for HMMER
! hmmpress ~/pfam/Pfam-A.hmm

In [ ]:
# Code to update the annotation system to actually run HMMER against the PFAM database
#. This code also stores the output in files, parses the results, and displays a visualization
import os
import sys
import subprocess
import pandas as pd
from Bio import SeqIO
import logging
import shutil
from tqdm import tqdm
import re

def annotate_unique_genes(unique_proteins_file, output_dir, pfam_db=None, cpu=4, evalue=1e-5):
    """
    Annotate unique genes using HMMER and Pfam database
    
    Parameters:
        unique_proteins_file (str): Path to FASTA file with protein sequences
        output_dir (str): Directory to save results
        pfam_db (str): Path to Pfam-A.hmm database (if None, tries to locate it)
        cpu (int): Number of CPU cores to use
        evalue (float): E-value threshold for significance
        
    Returns:
        tuple: (Path to HMMER output file, dictionary of parsed annotations)
    """
    # Setup logging
    logging.basicConfig(level=logging.INFO, 
                        format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Get base name for output files
    output_name = os.path.basename(unique_proteins_file).split('_')[0]
    hmmer_output = f"{output_dir}/{output_name}_pfam_annotations.txt"
    parsed_output = f"{output_dir}/{output_name}_pfam_parsed.tsv"
    
    # Check if HMMER is installed
    if not shutil.which("hmmscan"):
        logger.error("HMMER is not installed or not in PATH. Please install HMMER.")
        raise EnvironmentError("HMMER not found in PATH")
    
    # Find Pfam database if not provided
    if pfam_db is None:
        # Common locations for Pfam database
        possible_locations = [
            "/usr/local/share/pfam/Pfam-A.hmm",
            "/usr/share/pfam/Pfam-A.hmm",
            os.path.expanduser("~/pfam/Pfam-A.hmm"),
            "pfam/Pfam-A.hmm"
        ]
        
        for loc in possible_locations:
            if os.path.exists(loc):
                pfam_db = loc
                logger.info(f"Found Pfam database at {pfam_db}")
                break
                
        if pfam_db is None:
            logger.error("Pfam database not found. Please specify the path to Pfam-A.hmm")
            raise FileNotFoundError("Pfam database not found")
    
    # Check if Pfam database is pressed (indexed)
    if not os.path.exists(f"{pfam_db}.h3f"):
        logger.info("Pfam database needs to be pressed (indexed) first")
        press_cmd = f"hmmpress {pfam_db}"
        try:
            logger.info(f"Running: {press_cmd}")
            subprocess.run(press_cmd, shell=True, check=True)
        except subprocess.CalledProcessError as e:
            logger.error(f"Failed to index Pfam database: {e}")
            raise
    
    # Run HMMER against Pfam database
    cmd = f"hmmscan --domtblout {hmmer_output} -E {evalue} --cpu {cpu} {pfam_db} {unique_proteins_file}"
    
    try:
        logger.info(f"Running HMMER: {cmd}")
        process = subprocess.run(cmd, shell=True, check=True, 
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                                universal_newlines=True)
        logger.info("HMMER search completed successfully")
    except subprocess.CalledProcessError as e:
        logger.error(f"HMMER search failed: {e}")
        logger.error(f"STDERR: {e.stderr}")
        raise
    
    # Parse HMMER output (domtblout format)
    annotations = parse_hmmer_output(hmmer_output, parsed_output, evalue)
    
    logger.info(f"Functional annotation complete. Results saved to {hmmer_output}")
    logger.info(f"Parsed results saved to {parsed_output}")
    
    return hmmer_output, annotations

def parse_hmmer_output(hmmer_file, output_file, evalue_threshold=1e-5):
    """
    Parse HMMER domtblout file and extract domain annotations
    
    Parameters:
        hmmer_file (str): Path to HMMER domtblout output file
        output_file (str): Path to save parsed results
        evalue_threshold (float): E-value threshold for filtering
        
    Returns:
        dict: Dictionary with protein IDs as keys and lists of domain annotations as values
    """
    # Initialize results dictionary
    annotations = {}
    
    # Check if file exists and has content
    if not os.path.exists(hmmer_file) or os.path.getsize(hmmer_file) == 0:
        print(f"Warning: HMMER output file {hmmer_file} is empty or doesn't exist")
        return annotations
    
    # Check file content and print first few lines
    print(f"\nExamining HMMER output file: {hmmer_file}")
    with open(hmmer_file, 'r') as f:
        lines = f.readlines()
        data_lines = [line for line in lines if not line.startswith('#')]
        
        print(f"Total lines: {len(lines)}")
        print(f"Data lines (non-comment): {len(data_lines)}")
        
        if len(data_lines) > 0:
            print("First data line sample:")
            print(data_lines[0])
    
    # Parse the file
    results = []
    line_count = 0
    parsed_count = 0
    
    with open(hmmer_file, 'r') as f:
        for line in f:
            # Skip comment lines
            if line.startswith('#'):
                continue
                
            line_count += 1
            
            # Split the line on whitespace
            # HMMER domtblout format has fixed columns but variable spacing
            parts = line.strip().split()
            
            # Ensure we have enough parts for the required fields
            if len(parts) < 22:  # Minimum expected fields
                print(f"Warning: Line has fewer fields than expected: {len(parts)}")
                print(f"Line: {line.strip()}")
                continue
                
            try:
                # Construct a result with key fields
                # Note: domtblout format has target (hmm) first, then query (sequence)
                # Get target info (HMM)
                target_name = parts[0]
                target_accession = parts[1]
                
                # Get query info (sequence)
                query_name = parts[3]
                
                # Get e-values and scores
                evalue_fullseq = float(parts[6])
                score_fullseq = float(parts[7])
                evalue_dom = float(parts[11])
                score_dom = float(parts[12])
                
                # Get alignment positions
                hmm_from = int(parts[15])
                hmm_to = int(parts[16])
                ali_from = int(parts[17])
                ali_to = int(parts[18])
                
                # Get description (may be empty or multiple parts)
                description = " ".join(parts[22:]) if len(parts) >= 23 else ""
                
                # Filter by E-value
                if evalue_dom <= evalue_threshold:
                    row = {
                        'target_name': target_name,
                        'target_accession': target_accession,
                        'query_name': query_name,
                        'evalue_fullseq': evalue_fullseq,
                        'score_fullseq': score_fullseq,
                        'evalue_dom': evalue_dom,
                        'score_dom': score_dom,
                        'hmm_from': hmm_from,
                        'hmm_to': hmm_to,
                        'ali_from': ali_from,
                        'ali_to': ali_to,
                        'description': description
                    }
                    
                    results.append(row)
                    parsed_count += 1
                    
                    # Add to annotations dictionary
                    if query_name not in annotations:
                        annotations[query_name] = []
                    
                    domain_info = {
                        'domain': target_name,
                        'accession': target_accession,
                        'evalue': evalue_dom,
                        'score': score_dom,
                        'start': ali_from,
                        'end': ali_to,
                        'description': description
                    }
                    
                    annotations[query_name].append(domain_info)
                
            except (ValueError, IndexError) as e:
                print(f"Error parsing line: {line.strip()}")
                print(f"Error details: {str(e)}")
    
    print(f"Processed {line_count} data lines")
    print(f"Successfully parsed {parsed_count} domain hits")
    print(f"Found annotations for {len(annotations)} proteins")
    
    # Convert to DataFrame and save
    if results:
        df = pd.DataFrame(results)
        df = df.sort_values(by=['query_name', 'evalue_dom'])
        df.to_csv(output_file, sep='\t', index=False)
        print(f"Saved parsed results to {output_file}")
    else:
        print("No results to save to CSV")
        # Create empty file with headers
        with open(output_file, 'w') as f:
            headers = ['target_name', 'target_accession', 'query_name', 
                      'evalue_fullseq', 'score_fullseq', 'evalue_dom', 
                      'score_dom', 'hmm_from', 'hmm_to', 'ali_from', 
                      'ali_to', 'description']
            f.write('\t'.join(headers) + '\n')
    
    return annotations

def summarize_annotations(annotations, output_dir, protein_file):
    """
    Create a summary of domain annotations and visualize results
    
    Parameters:
        annotations (dict): Dictionary of annotations from parse_hmmer_output
        output_dir (str): Directory to save results
        protein_file (str): Path to the original protein FASTA file
        
    Returns:
        str: Path to summary file
    """
    import matplotlib.pyplot as plt
    from collections import Counter
    
    output_name = os.path.basename(protein_file).split('_')[0]
    summary_file = f"{output_dir}/{output_name}_annotation_summary.tsv"
    
    # Count domains
    all_domains = []
    for protein, domains in annotations.items():
        for domain in domains:
            all_domains.append(domain['domain'])
    
    domain_counts = Counter(all_domains)
    
    # Count proteins with annotations
    total_proteins = len(list(SeqIO.parse(protein_file, "fasta")))
    annotated_proteins = len(annotations)
    annotation_rate = (annotated_proteins / total_proteins) * 100 if total_proteins > 0 else 0
    
    # Write summary
    with open(summary_file, 'w') as f:
        f.write(f"Total proteins analyzed: {total_proteins}\n")
        f.write(f"Proteins with at least one domain: {annotated_proteins} ({annotation_rate:.1f}%)\n")
        f.write(f"Total domains found: {len(all_domains)}\n")
        f.write(f"Unique domain types: {len(domain_counts)}\n\n")
        
        f.write("Top 20 domains by frequency:\n")
        for domain, count in domain_counts.most_common(20):
            f.write(f"{domain}\t{count}\n")
    
    # Create visualization
    plt.figure(figsize=(12, 8))
    
    # Plot top 15 domains
    top_domains = dict(domain_counts.most_common(15))
    plt.bar(top_domains.keys(), top_domains.values())
    plt.xticks(rotation=45, ha='right')
    plt.title(f"Top 15 Protein Domains in {output_name}")
    plt.xlabel("Domain")
    plt.ylabel("Frequency")
    plt.tight_layout()
    
    plt.savefig(f"{output_dir}/{output_name}_domain_distribution.png", dpi=300)
    
    return summary_file

# Example usage
if __name__ == "__main__":
    # Set file paths
    protein_file = "unique_genes/NC_unique_proteins.faa". # this file was produced from your previous comparative genomics run
    output_dir = "annotation_results"
    
    # Check if the protein file exists
    if not os.path.exists(protein_file):
        print(f"ERROR: Protein file not found at {protein_file}")
        print("Please make sure the file exists at this location before running.")
        sys.exit(1)
    else:
        print(f"Found protein file: {protein_file}")
    
    # Validate and analyze the protein sequences
    try:
        sequences = list(SeqIO.parse(protein_file, "fasta"))
        seq_count = len(sequences)
        
        if seq_count == 0:
            print("ERROR: No valid sequences found in the file.")
            sys.exit(1)
            
        print(f"Found {seq_count} sequences in {protein_file}")
        
        # Calculate sequence statistics
        seq_lengths = [len(record.seq) for record in sequences]
        avg_length = sum(seq_lengths) / len(seq_lengths)
        min_length = min(seq_lengths)
        max_length = max(seq_lengths)
        
        print(f"Sequence statistics:")
        print(f"  Average length: {avg_length:.1f} amino acids")
        print(f"  Minimum length: {min_length} amino acids")
        print(f"  Maximum length: {max_length} amino acids")
        
        # Print a few example sequences
        print("\nExample sequences from Unique genes file:")
        for i, record in enumerate(sequences):
            if i < 3:  # Show first 3 sequences
                print(f">{record.id}")
                print(f"  Length: {len(record.seq)} aa")
                print(f"  Sequence: {record.seq[:50]}..." if len(record.seq) > 50 else f"  Sequence: {record.seq}")
            else:
                break
    except Exception as e:
        print(f"ERROR: Failed to analyze sequences: {str(e)}")
        sys.exit(1)
        
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Run annotation with less stringent E-value
    try:
        print(f"\nStarting annotation of {protein_file}...")
        print(f"Using less stringent E-value threshold (1e-3) to increase sensitivity")
        
        hmmer_output, annotations = annotate_unique_genes(
            protein_file, 
            output_dir,
            pfam_db=None,  # Auto-detect
            cpu=4,
            evalue=1e-3  # Less stringent threshold
        )
        
        # Check if annotations were found
        if not annotations:
            print("\nNo annotations found with E-value threshold of 1e-3.")
            print("Let's try with an even less stringent threshold (1e-2)...")
            
            hmmer_output, annotations = annotate_unique_genes(
                protein_file, 
                output_dir,
                pfam_db=None,
                cpu=4,
                evalue=1e-2  # Very permissive threshold
            )
            
        # Check HMMER output file content even if no annotations
        if os.path.exists(hmmer_output) and os.path.getsize(hmmer_output) > 0:
            print("\nExamining raw HMMER output:")
            with open(hmmer_output, "r") as f:
                lines = f.readlines()
                header_lines = [line for line in lines if line.startswith('#')]
                data_lines = [line for line in lines if not line.startswith('#')]
                
                print(f"  Header lines: {len(header_lines)}")
                print(f"  Data lines: {len(data_lines)}")
                
                if data_lines:
                    print("\nFirst few matches (if any):")
                    for i, line in enumerate(data_lines):
                        if i < 3:
                            print(f"  {line.strip()}")
                        else:
                            break
                else:
                    print("  No matches found in HMMER output.")
                    
            # If still no annotations, try a different approach
            if not annotations:
                print("\nNo Pfam domains found even with permissive threshold.")
                print("Suggestions:")
                print("1. Try a different domain database (e.g., TIGRFAMs, CDD)")
                print("2. Consider using sequence similarity search (BLAST) instead")
                print("3. Check if these sequences are likely to be known proteins")
        
    except Exception as e:
        print(f"ERROR: Annotation failed: {str(e)}")
        sys.exit(1)
    
    # Generate summary and visualizations
    if annotations:
        summary_file = summarize_annotations(annotations, output_dir, protein_file)
        print(f"Annotation summary saved to {summary_file}")
    else:
        print("No annotations found.")

In [ ]:
# You should see your result files in annotation_results/ subdirectory, files: NC_pfam_annotations.txt 
#.  and NC_pfam_parsed.csv
# You will also see the output visualization graph

In [ ]:
# If you run into problems, you can try some of these types of prompts to fix / upgrade your code:
# Write code to install the pram database for me
# update the above code so the input file is in unique_genes/NC_unique_proteins.faa
# It looks like there is an issue with the parsing function
#. You can also paste any errors you get back into your AI tool and iterate until it is working

In [ ]:
## End of Notebook ##